In [1]:
import random
from six.moves import cPickle as pickle
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.patches import Circle
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rc('text.latex', preamble='\\usepackage{color}')
from PIL import Image
import numpy as np
from scipy.misc import comb
from math import log, exp, sqrt
from collections import Counter, OrderedDict
import seaborn as sns
sns.set_context("paper")
# sns.set_context('talk')
import string
import json
import pickle
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
%matplotlib inline
# COLORS = ["#808080", "#3498db", "#0000b3", "#ffd480", "#e74c3c"]
blues = sns.color_palette('Blues', 10)
reds = sns.color_palette('Reds')
COLORS = ["#808080", reds[2], blues[5], blues[9]]
# sns.palplot(sns.color_palette(COLORS))

# from multiprocessing import set_start_method
# try:
#     set_start_method('spawn')
# except RuntimeError:
#     pass

## Config data

In [2]:
# Constants
itow = json.load(open('../data/coco/cocotalk.json', 'r'))['ix_to_word']
WORDS = {int(k): v for k, v in itow.items()}
if 0 not in WORDS:
    WORDS[0] = 'EOS'
MATCH = {v: k for k,v in WORDS.items()}
del itow
FF = pickle.load(open('../data/coco/idf_coco_01.pkl', 'rb'))
F = pickle.load(open('cooc_freq.pkl', 'rb'))[0]
sim_coco = pickle.load(open('../data/Glove/glove_coco_d300_w15_similarities.pkl','rb'))
G2_coco = pickle.load(open('../data/Glove/glove_coco_d300_w15_matrix_tsne2D.pkl', 'rb'))

A = json.load(open('../data/coco/captions_train2014.json', 'r'))['annotations']
caps = {}
for a in A:
    if a['image_id'] in caps:
        caps[a['image_id']].append(a['caption'][:-1].lower().split())
    else:
        caps[a['image_id']] = [a['caption'][:-1].lower().split()]
del A


A = json.load(open('../data/coco/captions_val2014.json', 'r'))['annotations']
caps_val = {}
for a in A:
    if a['image_id'] in caps_val:
        caps_val[a['image_id']].append(a['caption'][:-1].lower().split())
    else:
        caps_val[a['image_id']] = [a['caption'][:-1].lower().split()]
del A

In [ ]:
def attention_visualization(root, image_name, caption, alphas):
    image = Image.open(os.path.join(root, image_name))
    image = image.resize([224, 224], Image.LANCZOS)
    plt.subplot(4,5,1)
    plt.imshow(image)
    plt.axis('off')
    
    words = caption[1:]
    for t in range(len(words)):
        if t > 18:
            break
        plt.subplot(4, 5, t+2)
        plt.text(0, 1, '%s'%(words[t]) , color='black', backgroundcolor='white', fontsize=8)
        plt.imshow(image)
        # print alphas
        alp_curr = alphas[t, :].view(14, 14)
        alp_img = skimage.transform.pyramid_expand(alp_curr.numpy(), upscale=16, sigma=20)
        plt.imshow(alp_img, alpha=0.85)
        plt.axis('off')
    plt.show()
    
